In [3]:
# Import of the librairie we'll use in this project
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [66]:
notebook_file = Path("EDA_exploration.ipynb")
notebook_file.resolve

<bound method Path.resolve of WindowsPath('EDA_exploration.ipynb')>

In [56]:
# defining the path of the dataset for the import
# racine du projet
project_root = Path("__file__").resolve().parent.parent
# the data to load
insurance_path = project_root / "data" / "raw" / "insurance.csv"

In [57]:
# load the data in the dataframe
df = pd.read_csv(filepath_or_buffer=insurance_path, sep=",")

In [58]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [65]:
my_notebook = Path("EDA_exploration.ipynb")
print(my_notebook)

EDA_exploration.ipynb
